In [ ]:
#import the required libraries
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.ticker as mtick  
import matplotlib.pyplot as plt
%matplotlib inline

import os
print("Input data file:  ",os.listdir("../input"))

sns.set(style = 'white')

creating data frame and its copy

In [ ]:
diabetes_data_original = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
diabetes_data = diabetes_data_original.copy()

printing names of columns

In [ ]:
print("\n".join(diabetes_data.columns.values))

fast look at the first 10 rows

In [ ]:
diabetes_data.head(10)

how big is dataset - it has 9 columns (variables) and 768 lines (records)
from data info I know that outcome is dependant variable y and others are independent variables x

In [ ]:
diabetes_data.shape

In [ ]:
diabetes_data.dtypes

from result of types I see that everything is integer or float, however I saw 0 values in some variables, Outcome is categorical variable (0,1) and zeros in pregnancies (could be)


In [ ]:
diabetes_data.describe()

**First findings**
I see that 75% of women had 0-6 pregnancies, however other had more (max is 17 lol). 

Next finding is that BloodPressure, SkinThickness, Insulin, BMI have 0 min value. It means I would replace zeros with mean. Glucose could be?
From data description I know that there should be women with 21 or more years (it is correct)

Also if outcome is 1, there is huge glucose and insulin. It means that it makes sense to split data based on outcome and replace 0 by mean of group. Second observation could be replacing 0 by mean of whole dataset and compare. 

From average I see that most of women are young (mean 33 and 75%quantile 41 years), then mean of outcome is 0,35 so it should be more women without diabetes. 

In [ ]:
obesity = diabetes_data.loc[diabetes_data ['BMI'] > 30]
obesity_count = obesity['BMI'].count()
print('There are '+ str(obesity_count) + ' obese patients according to CDC.')

In [ ]:
oldest_patients = diabetes_data.loc[diabetes_data ['Age'] > np.quantile(diabetes_data['Age'], 0.75)]
BP_mean_of_oldest = oldest_patients['BloodPressure'].mean()
print('Mean of blood pressure of 25% oldest patients is '+ str(round(BP_mean_of_oldest, 3)) +'.')

# DATA CLEANING AND IMPUTATION BASED ON OUTPUT

As we have no missing values, we have to replace only zeros. I will try method by imputing zero values of Blood Pressure by mean according to Outcome. 
Firstly, I filter dataset into two groups - first with outcome 0, second with outcome 1. 
Mean is calculated from data of group, means that they are not equal.

In [ ]:
group0 = diabetes_data.loc[diabetes_data ['Outcome'] == 0]
group1 = diabetes_data.loc[diabetes_data ['Outcome'] == 1]
mean_group0 = group0['BloodPressure'].mean()
mean_group1 = group1['BloodPressure'].mean()
print('group0 - ' + str(group0['Outcome'].count()) + ' cases, ' + 'Blood Pressure mean of group 0 is ' + str(mean_group0))
print('group1 - ' + str(group1['Outcome'].count()) + ' cases, ' + 'Blood Pressure mean of group 1 is ' + str(mean_group1))
#print(group0.head())

In [ ]:
group0.loc[group0 ['BloodPressure'] == 0 ,'BloodPressure'] = mean_group0
group1.loc[group1 ['BloodPressure'] == 0 ,'BloodPressure'] = mean_group1

Replacing 0 from blood presure by mean of group0 and group1. There is no 0 - can check it in describe ->min is now 24 for blood pressure.

Append of two groups together into one variable but zero values for BP are replaced by mean of group 1 or group 0.

In [ ]:
clean_data = group0.append(group1)
clean_data.describe()

In [ ]:
clean_data.shape
sns.lmplot(x="Age",y="Glucose", data=clean_data.head(700), hue="Outcome", aspect=2/1, height=7, fit_reg=False)

I see that most outcome with no diabetes are young and have low glucose. (it makes sense). Patients with diabetes have higher level of glucose, so there could be connection between Glucose and Outcome.

In [ ]:
sns.countplot(data=clean_data, x='Age', hue='Outcome')

Distribution by age is as expected - most young women have no diabetes. 

In [ ]:
sns.countplot(data=clean_data, x='Pregnancies', hue='Outcome')

One more interesting fact is that most women with no pregnancies or only a few, have no diabetes.

In [ ]:
sns.countplot(data=clean_data, x='Outcome', hue='Outcome')

In [ ]:
sns.scatterplot(data=clean_data, x="Insulin", y="Glucose")

I see that lot of patiens have 0 insulin so should be imputed however the data will be distorted.

In [ ]:
sns.scatterplot(data=clean_data, x="Age", y="Pregnancies")

no relationship

In [ ]:
sns.scatterplot(data=clean_data, x="BloodPressure", y="Glucose")

no relationship

In [ ]:
sns.scatterplot(data=clean_data, x="BMI", y="Glucose")

no relationship

In [ ]:
correlations = clean_data.corr()['Outcome']
plt.figure(figsize=(20,10))
correlations.drop('Outcome').sort_values(ascending = False).plot(kind='bar')

# The most important predictors of diabetes is Glucose, BMI, Age. These specifications have the highest correlation with Outcome. Especially Glucose has positive correlation almost 0,5.

In [ ]:
correlations = clean_data.corr()['Insulin']
plt.figure(figsize=(20,10))
correlations.drop('Insulin').sort_values(ascending = False).plot(kind='bar')

In [ ]:
Tot = sns.kdeplot(clean_data.Age[(clean_data["Outcome"] == 0) ],
                color="Red", shade = True)
Tot = sns.kdeplot(clean_data.Age[(clean_data["Outcome"] == 1) ],
                ax =Tot, color="Blue", shade= True)
Tot.legend(["No diabetes","Have diabetes"],loc='upper right')
Tot.set_ylabel('Density')
Tot.set_xlabel('Age')
Tot.set_title('Age by outcome')

In [ ]:
Tot = sns.kdeplot(clean_data.Glucose[(clean_data["Outcome"] == 0) ],
                color="Red", shade = True)
Tot = sns.kdeplot(clean_data.Glucose[(clean_data["Outcome"] == 1) ],
                ax =Tot, color="Blue", shade= True)
Tot.legend(["No diabetes","Have diabetes"],loc='upper right')
Tot.set_ylabel('Density')
Tot.set_xlabel('Glucose')
Tot.set_title('Glucose by outcome')